Documentation at https://trio.readthedocs.io/en/latest/tutorial.html

In [1]:
# A regular function
def regular_double(x):
    return 2 * x

# An async function
async def async_double(x):
    return 2 * x

In [2]:
async def print_double(x):
    print(await async_double(x))   # <-- OK!

In [4]:
%pip install trio

     |████████████████████████████████| 327kB 522kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import trio

async def async_double(x):
    return 2 * x

trio.run(async_double, 3)  # returns 6

6

In [8]:
import trio

async def double_sleep(x):
    await trio.sleep(2 * x)

trio.run(double_sleep, 3)

print("Waiting")

Waiting


In [15]:
%pip install --pre jupyterlab

Note: you may need to restart the kernel to use updated packages.


In [7]:
zae = 4

In [13]:
import asyncio
import random
import time


async def worker(name, queue):
    while True:
        # Get a "work item" out of the queue.
        sleep_for = await queue.get()

        # Sleep for the "sleep_for" seconds.
        await asyncio.sleep(sleep_for)

        # Notify the queue that the "work item" has been processed.
        queue.task_done()

        print(f'{name} has slept for {sleep_for:.2f} seconds')

In [14]:
# Create a queue that we will use to store our "workload".
queue = asyncio.Queue()

# Generate random timings and put them into the queue.
total_sleep_time = 0
for _ in range(20):
    sleep_for = random.uniform(0.05, 1.0)
    total_sleep_time += sleep_for
    queue.put_nowait(sleep_for)

# Create three worker tasks to process the queue concurrently.
tasks = []
for i in range(3):
    task = asyncio.create_task(worker(f'worker-{i}', queue))
    tasks.append(task)

# Wait until the queue is fully processed.
started_at = time.monotonic()
await queue.join()
total_slept_for = time.monotonic() - started_at

# Cancel our worker tasks.
for task in tasks:
    task.cancel()
# Wait until all worker tasks are cancelled.
await asyncio.gather(*tasks, return_exceptions=True)

print('====')
print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
print(f'total expected sleep time: {total_sleep_time:.2f} seconds')

worker-0 has slept for 0.06 seconds
worker-2 has slept for 0.47 seconds
worker-1 has slept for 0.53 seconds
worker-2 has slept for 0.19 seconds
worker-2 has slept for 0.06 seconds
worker-0 has slept for 0.73 seconds
worker-1 has slept for 0.28 seconds
worker-2 has slept for 0.20 seconds
worker-0 has slept for 0.60 seconds
worker-1 has slept for 0.60 seconds
worker-2 has slept for 0.60 seconds
worker-0 has slept for 0.24 seconds
worker-1 has slept for 0.65 seconds
worker-0 has slept for 0.75 seconds
worker-2 has slept for 0.90 seconds
worker-1 has slept for 0.44 seconds
worker-1 has slept for 0.25 seconds
worker-2 has slept for 0.55 seconds
worker-0 has slept for 0.64 seconds
worker-1 has slept for 0.65 seconds
====
3 workers slept in parallel for 3.40 seconds
total expected sleep time: 9.41 seconds
